In [1]:
import gym 
import numpy as np

from function_approximators.function_approximators import NeuralNetwork, LinearModel
from train_utils import train

from agents.agents import DQNAgent, LinearAgent

RENDER = False
env = gym.make("CartPole-v1")

In [2]:
function_approximators = [NeuralNetwork, LinearModel]
agents = [DQNAgent, LinearAgent]

In [7]:

# for lr in np.linspace(0.001,0.07,20):

returns = []
n_seeds=10

for i in range(n_seeds):

    print(f"\n Run: {i+1} \n")

    # print(f"Learning rate: {lr}")
    # Linear Config
    CONFIG = {
        "episode_length": 200,
        "max_timesteps": 30000,
        "max_time": 30 * 60,
        "eval_freq": 2000, 
        "eval_episodes": 5,
        "learning_rate": 0.03,
        "target_update_freq": 50,
        "batch_size": 32,
        "gamma": 0.99,
        "buffer_capacity": int(1e7),
        "plot_loss": False,
        "epsilon": 1,
        "max_steps": 200,
        "poly_degree": 2,
        "max_deduct": 0.97,
        "decay": 1,
        "lr_step_size": 1000,
        "lr_gamma": 0.95,
    }
    r, _ = train(env, 
            CONFIG, 
            fa=function_approximators[1], 
            agent = agents[1], 
            render=RENDER)
    env.close()
    returns.append(r[-1])

print(f"Mean returns: {np.mean(returns)}")
print(f"Std returns: {np.std(returns)}")

  0%|          | 91/30000 [00:00<00:36, 828.50it/s]
 Run: 1 

  7%|▋         | 2105/30000 [00:02<00:40, 685.34it/s]Evaluation at timestep 2009 returned a mean returns of 9.6
Learning rate - 0.029699999999999997
 14%|█▎        | 4089/30000 [00:05<00:45, 568.65it/s]Evaluation at timestep 4000 returned a mean returns of 22.0
Learning rate - 0.029108969999999998
 15%|█▌        | 4528/30000 [00:06<00:36, 695.49it/s]


RuntimeError: Sizes of tensors must match except in dimension 1. Got 2 and 32 in dimension 0 (The offending index is 1)